In [ ]:
# A collection of utility functions in no particular order
# Version : 1.0
# Date : September 14, 2016
# Author: Sergey Antopolsky

In [1]:
import numpy as np
import pandas as pd

In [ ]:
def new_line_string(s,limit=20):
    """
    Takes a long string and attempts to make new lines between words. If
    cannot get between words, break the word and puts a "-" sign in the end
    of the line. Useful for placing compact notes on figures.
    
    20160914 by Sergey Antopolskiy
    """
    if type(s) == str:
        changed_s = ''
        while len(s)>limit:
            if s[limit-1]==' ':
                changed_s += s[:limit] + '\n'
                s = s[limit:]
            elif s[limit]==' ':
                changed_s += s[:limit] + '\n'
                s = s[limit+1:]
            else:
                changed_s += s[:limit] + '-\n'
                s = s[limit:]
        changed_s += s
        return changed_s
    else:
        return ''

In [ ]:
def find_stim_without_another_before_it(motor_trace_data,trigs,options,threshold=1000):
    """
    Returns index of the trigger which is associates with a stimulus which
    doesn't have any motor movement right before it appeared. Helpful for 
    plotting example of a stimulus i the PSTH figure.
    
    20160915 by Sergey Antopolskiy
    """
    motor_fs = motor_trace_data['fs']
    for (i,trig) in enumerate(trigs):
        before_stim_trace = motor_trace_data['data'][int((trigs.iloc[i]-options['before'])*motor_fs):
                                       int((trigs.iloc[i])*motor_fs)]
        if sum(np.abs(before_stim_trace))<threshold:
            break
    return i

In [ ]:
def sanity_check(x,value,direction='not_eq'):
    """
    Raise an error if x!=value. If direction = 'eq', raises an error if x==value.
    
    20160927 by Sergey Antopolskiy
    """
    if direction == 'eq':
        if x == value:
            raise
    elif direction == 'not_eq':
        if x != value:
            raise
    else:
        print("Wrong direction parameter, did nothing.")

In [ ]:
def prop_true(array, ax=0, return_len=0):
    if return_len:
        return np.sum(array,ax)/array.shape[ax], array.shape[ax]
    else:
        return np.sum(array,ax)/array.shape[ax]

In [ ]:
import numpy.random as npr

def bootstrap(data, num_samples, statistic, alpha=0):
    """
    Returns resampled values and if alpha is provided, estimates 
    two-tailed (1-alpha) confidence interval for statistic.
    
    Modified from: http://people.duke.edu/~ccc14/pcfb/analysis.html
    """
    n = len(data)
    idx = npr.randint(0, n, (num_samples, n))
    samples = data[idx]
    resampled_values = statistic(samples,1)
    stat = np.sort(resampled_values)
    if alpha != 0:
        return (stat[int((alpha/2.0)*num_samples)],
        stat[int((1-alpha/2.0)*num_samples)]), resampled_values
    else:
        return resampled_values
        
def bootstrap_compare(values_1, values_2, statistics, num_samples = 1000):
    """
    Performs a comparison of resampled statistics between 2 vectors. Subtracts each resampled
    values of second group from each of the first group and returns % of differences which are
    below 0; it corresponds to p-value of resampling test.
    """
    resampled_1 = bootstrap(values_1,num_samples,statistics)
    resampled_2 = bootstrap(values_2,num_samples,statistics)
    
    stat = np.array([a-b for a in resampled_1 for b in resampled_2])
    return sum(stat<0)/len(stat)

In [ ]:
def pandas_series_tuple_unpack(series,column_names):
    """
    Converts a Series of tuples to DataFrame with single values.
    
    """
    return pd.DataFrame(series.tolist(), columns=column_names, index=series.index)